In [1]:
# Atualização das bibliotecas
!pip install --upgrade keras tensorflow

Requirement already up-to-date: tensorflow in c:\programdata\anaconda3\lib\site-packages (2.3.0)


In [3]:
# Download do dataset de Treinamento
!gdown https://drive.google.com/uc?id=1xQqTSUA-celn_KRB9Qpx77g54KR0WHaH
!unzip Leishmania_Treinamento.zip
!rm Leishmania_Treinamento.zip

'gdown' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.
'unzip' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.
'rm' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.


In [5]:
tf.version

<module 'tensorflow._api.v2.version' from 'c:\\users\\jhon-wilker\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14401623758635889189
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10082065090060358009
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16712326371613154626
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
# Fixando seeds pra ter resultados mais consistentes
import numpy as np
import tensorflow as tf
import random as python_random

from distutils.version import LooseVersion
import warnings
import tensorflow as tf

!nvidia-smi

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
print('Found GPU at: {}'.format(device_name))
    
np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

Fri Sep 18 14:59:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.15       Driver Version: 460.15       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   46C    P8    12W / 100W |    610MiB /  4096MiB |     11%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

c:\users\jhon-wilker\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please ensure you have installed TensorFlow correctly
  


In [7]:
!nvidia-smi

Fri Sep 18 15:06:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.15       Driver Version: 460.15       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   45C    P8    11W / 100W |    629MiB /  4096MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [6]:
# Imports
from os import listdir
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
# Inicialização do modelo
classificador = Sequential()

tamanho_imagem = (256, 256)
# Carregamento da arquitetura
transfer = ResNet152V2(include_top = False, input_shape = (tamanho_imagem[0], 
                                                           tamanho_imagem[1], 3))
for camada in transfer.layers:
  camada.trainable = True

classificador.add(transfer)

# Saída do modelo
classificador.add(Flatten())
classificador.add(Dense(1, activation = 'sigmoid'))

classificador.compile(optimizer = Adam(learning_rate = 0.00001),
               loss = 'binary_crossentropy', metrics = ['accuracy'])

batch = 10

# Nenhuma forma de Augmentation
# Rescale é apenas normalização
datagen = ImageDataGenerator(rescale=1./255)


grupo_treinamento = datagen.flow_from_directory('Leishmania_Treinamento/treinamento', 
                                                target_size=tamanho_imagem, 
                                                batch_size=batch, 
                                                class_mode='binary')

grupo_validacao = datagen.flow_from_directory('Leishmania_Treinamento/validacao', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')

classificador.fit(grupo_treinamento, epochs=5, validation_data=grupo_validacao)

234553344/234545216 [==============================] - 69s 0us/step
Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Epoch 1/5
32/32 [==============================] - 373s 12s/step - loss: 0.7039 - accuracy: 0.6656 - val_loss: 0.6014 - val_accuracy: 0.7000
Epoch 2/5
32/32 [==============================] - 358s 11s/step - loss: 0.1546 - accuracy: 0.9563 - val_loss: 0.5610 - val_accuracy: 0.7000
Epoch 3/5
32/32 [==============================] - 378s 12s/step - loss: 0.0636 - accuracy: 0.9844 - val_loss: 0.4206 - val_accuracy: 0.7500
Epoch 4/5
32/32 [==============================] - 373s 12s/step - loss: 0.0477 - accuracy: 0.9906 - val_loss: 0.3658 - val_accuracy: 0.8000
Epoch 5/5
32/32 [==============================] - 376s 12s/step - loss: 0.0490 - accuracy: 0.9906 - val_loss: 0.3172 - val_accuracy: 0.8750


In [ ]:
# Download do dataset de Teste
!gdown https://drive.google.com/uc?id=1tTBldPLMQk8jLLDjBaxwbl_FXpYNS0Ux
!unzip Leishmania_Teste.zip
!rm Leishmania_Teste.zip

In [11]:
# Funções

# Preprocessamento das imagens
def carregar_imagem(arquivo):
  img = image.load_img(arquivo, target_size=tamanho_imagem)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  return img

# Carregar imagens em uma lista
def predicao_teste(pasta_teste):
  imagens = []
  for pasta in listdir(pasta_teste):
    for arquivo in listdir(pasta_teste + '/' + pasta):
      imagens.append(carregar_imagem(pasta_teste + pasta + '/' + arquivo))
  
  return imagens

In [9]:
imagens = np.vstack(predicao_teste('Leishmania_Teste/'))
respostas_modelo = classificador.predict_classes(imagens)
respostas_teste = datagen.flow_from_directory('Leishmania_Teste/', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')
print('Respostas do modelo\n' + str(np.concatenate(respostas_modelo)))
print('Respostas reais\n' + str(respostas_teste.classes))

# Matriz de confusão
matriz_de_confusao = confusion_matrix(respostas_teste.classes, respostas_modelo)

# Metricas
# Marvin pediu especificamente pra gente avaliar os valores de sensibilidade e
# especificidade
TrueNegative, FalsePositive, FalseNegative, TruePositive = matriz_de_confusao.ravel()

# Precisão
print('Precisao ' + str((TruePositive + TrueNegative) / len(respostas_modelo)))

# Sensibilidade
print('Sensibilidade ' + str(TruePositive / (TruePositive + FalseNegative)))

# Especificidade
print('Especificidade ' + str(TrueNegative / (FalsePositive + TrueNegative)))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Found 44 images belonging to 2 classes.
Respostas do modelo
[0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0
 0 0 0 0 0 0 1]
Respostas reais
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]
Precisao 0.6363636363636364
Sensibilidade 0.4090909090909091
Especificidade 0.8636363636363636


In [12]:
# Salvar modelo treinado em um arquivo
classificador.save('modelo_leishmania.h5')